In [1]:
import numpy as np
import pandas as pd
import seaborn
import matplotlib.pyplot as plt

In [8]:
calendar = pd.read_csv("calendar08.csv",sep=",")
calendar

FileNotFoundError: [Errno 2] File b'calendar08.csv' does not exist: b'calendar08.csv'

In [19]:
calendar = calendar.sort_values(by=["listing_id", "date"])
calendar = calendar.reset_index()
calendar

,index,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,222,2577,2020-08-25,t,$125.00,$125.00,3.0,1125.0
1,223,2577,2020-08-26,t,$125.00,$125.00,3.0,1125.0
2,224,2577,2020-08-27,t,$125.00,$125.00,3.0,1125.0
3,225,2577,2020-08-28,t,$125.00,$125.00,3.0,1125.0
4,226,2577,2020-08-29,t,$125.00,$125.00,3.0,1125.0
...,...,...,...,...,...,...,...,...
24667027,24575645,45070724,2021-08-21,t,$67.00,$67.00,1.0,1125.0
24667028,24575646,45070724,2021-08-22,t,$67.00,$67.00,1.0,1125.0
24667029,24575647,45070724,2021-08-23,t,$67.00,$67.00,1.0,1125.0
24667030,24575648,45070724,2021-08-24,t,$67.00,$67.00,1.0,1125.0


In [20]:
calendar.shape

(24667032, 8)

In [21]:
del calendar['price']
del calendar['index']
del calendar['adjusted_price']
calendar

,listing_id,date,available,minimum_nights,maximum_nights
0,2577,2020-08-25,t,3.0,1125.0
1,2577,2020-08-26,t,3.0,1125.0
2,2577,2020-08-27,t,3.0,1125.0
3,2577,2020-08-28,t,3.0,1125.0
4,2577,2020-08-29,t,3.0,1125.0
...,...,...,...,...,...
24667027,45070724,2021-08-21,t,1.0,1125.0
24667028,45070724,2021-08-22,t,1.0,1125.0
24667029,45070724,2021-08-23,t,1.0,1125.0
24667030,45070724,2021-08-24,t,1.0,1125.0


In [22]:
data = calendar
data.shape

(24667032, 5)

In [23]:
def groupDate(group):
    return [group.iloc[0]['date'], group.iloc[-1]['date'], group.shape[0], str(group.iloc[0]['listing_id']),  group.iloc[0]['available'],  group.iloc[0]['minimum_nights'],  group.iloc[0]['maximum_nights']]

In [24]:
adj_check = (data.available != data.available.shift()).cumsum()
newData = pd.DataFrame(data.groupby(['listing_id','available',adj_check], as_index=False, sort=False).apply(groupDate))
newData[['start','end','num_day','listing_id','available','minimum_nights','maximum_nights']] = pd.DataFrame(newData[0].to_list(), index= newData.index)
newData["start"] = pd.to_datetime(newData["start"])
newData["end"] = pd.to_datetime(newData["end"])
del newData[0]
newData

start        end  num_day listing_id  \
listing_id available available                                             
2577       t         1         2020-08-25 2021-08-24      365       2577   
3109       t         1         2020-08-26 2021-08-25      365       3109   
4890       t         1         2020-08-23 2020-08-23        1       4890   
           f         2         2020-08-24 2020-08-24        1       4890   
           t         3         2020-08-25 2020-08-28        4       4890   
...                                   ...        ...      ...        ...   
45069732   t         171435    2020-09-23 2021-08-25      337   45069732   
45070724   f         171436    2020-08-26 2020-08-26        1   45070724   
           t         171437    2020-08-27 2020-08-30        4   45070724   
           f         171438    2020-08-31 2020-09-03        4   45070724   
           t         171439    2020-09-04 2021-08-25      356   45070724   

                               available  minimum_nights  maximum_nights  
listing_id available available                                            
2577       t         1                 t             3.0          1125.0  
3109       t         1                 t             2.0            30.0  
4890       t         1                 t             4.0           120.0  
           f         2                 f             4.0           120.0  
           t         3                 t             4.0           120.0  
...                                  ...             ...             ...  
45069732   t         171435            t             1.0            25.0  
45070724   f         171436            f             1.0          1125.0  
           t         171437            t             1.0          1125.0  
           f         171438            f             1.0          1125.0  
           t         171439            t             1.0          1125.0  

[218237 rows x 7 columns]

In [25]:
newData.shape #(309585, 8)

(218237, 7)

In [26]:
def label(row):
    try:
        if row['available'] == 'f':
            if row['num_day'] < row['minimum_nights'] or row['num_day'] > row['maximum_nights']:
                return 'closed > max nights'
            else:
                if row['num_day'] > 21:
                    return 'closed > 21 days'
                elif row['num_day'] > 14:
                    return 'closed > 14 days'
                elif row['num_day'] > 7:
                    return 'closed > 7 days'
                return 'closed < 7 days'
        else:
            return 'available'
    except:
        return 'error'

In [27]:
newData['label'] = newData.apply(label, axis=1)
newData

start        end  num_day listing_id  \
listing_id available available                                             
2577       t         1         2020-08-25 2021-08-24      365       2577   
3109       t         1         2020-08-26 2021-08-25      365       3109   
4890       t         1         2020-08-23 2020-08-23        1       4890   
           f         2         2020-08-24 2020-08-24        1       4890   
           t         3         2020-08-25 2020-08-28        4       4890   
...                                   ...        ...      ...        ...   
45069732   t         171435    2020-09-23 2021-08-25      337   45069732   
45070724   f         171436    2020-08-26 2020-08-26        1   45070724   
           t         171437    2020-08-27 2020-08-30        4   45070724   
           f         171438    2020-08-31 2020-09-03        4   45070724   
           t         171439    2020-09-04 2021-08-25      356   45070724   

                               available  minimum_nights  maximum_nights  \
listing_id available available                                             
2577       t         1                 t             3.0          1125.0   
3109       t         1                 t             2.0            30.0   
4890       t         1                 t             4.0           120.0   
           f         2                 f             4.0           120.0   
           t         3                 t             4.0           120.0   
...                                  ...             ...             ...   
45069732   t         171435            t             1.0            25.0   
45070724   f         171436            f             1.0          1125.0   
           t         171437            t             1.0          1125.0   
           f         171438            f             1.0          1125.0   
           t         171439            t             1.0          1125.0   

                                              label  
listing_id available available                       
2577       t         1                    available  
3109       t         1                    available  
4890       t         1                    available  
           f         2          closed > max nights  
           t         3                    available  
...                                             ...  
45069732   t         171435               available  
45070724   f         171436         closed < 7 days  
           t         171437               available  
           f         171438         closed < 7 days  
           t         171439               available  

[218237 rows x 8 columns]

In [28]:
newData['label'].value_counts()*100/newData.shape[0]

available              40.600356
closed > 21 days       20.854850
closed < 7 days        18.099589
closed > max nights    16.596636
closed > 7 days         2.898225
closed > 14 days        0.950343
Name: label, dtype: float64

In [29]:
newData[newData['listing_id'] == '93287']

start        end  num_day listing_id  \
listing_id available available                                             
93287      f         644       2020-08-23 2020-09-04       13      93287   
           t         645       2020-09-05 2020-10-01       27      93287   
           f         646       2020-10-02 2020-10-05        4      93287   
           t         647       2020-10-06 2021-03-12      158      93287   
           f         648       2021-03-13 2021-03-19        7      93287   
           t         649       2021-03-20 2021-04-20       32      93287   
           f         650       2021-04-21 2021-04-24        4      93287   
           t         651       2021-04-25 2021-05-12       18      93287   
           f         652       2021-05-13 2021-05-17        5      93287   
           t         653       2021-05-18 2021-08-22       97      93287   

                               available  minimum_nights  maximum_nights  \
listing_id available available                                             
93287      f         644               f             3.0          1125.0   
           t         645               t             3.0          1125.0   
           f         646               f             3.0          1125.0   
           t         647               t             3.0          1125.0   
           f         648               f             3.0          1125.0   
           t         649               t             3.0          1125.0   
           f         650               f             3.0          1125.0   
           t         651               t             3.0          1125.0   
           f         652               f             3.0          1125.0   
           t         653               t             3.0          1125.0   

                                          label  
listing_id available available                   
93287      f         644        closed > 7 days  
           t         645              available  
           f         646        closed < 7 days  
           t         647              available  
           f         648        closed < 7 days  
           t         649              available  
           f         650        closed < 7 days  
           t         651              available  
           f         652        closed < 7 days  
           t         653              available

In [30]:
newData.to_csv('c09.csv')

In [9]:
import numpy as np
import pandas as pd

def groupDate(group):
    return [group.iloc[0]['date'], group.iloc[-1]['date'], str(group.iloc[0]['listing_id']),  group.iloc[0]['available'],group.shape[0],  group.iloc[0]['minimum_nights'],  group.iloc[0]['maximum_nights']]

def OptimizeCalendar(filename):
    calendar = pd.read_csv(filename,sep=",")

    calendar = calendar.sort_values(by=["listing_id", "date"])
    calendar = calendar.reset_index()

    del calendar['price']
    del calendar['index']
    del calendar['adjusted_price']

    adj_check = (calendar.available != calendar.available.shift()).cumsum()
    newData = pd.DataFrame(calendar.groupby(['listing_id','available',adj_check], as_index=False, sort=False).apply(groupDate))
    newData[['start','end','listing_id','available','num_day','minimum_nights','maximum_nights']] = pd.DataFrame(newData[0].to_list(), index= newData.index)
    newData["start"] = pd.to_datetime(newData["start"])
    newData["end"] = pd.to_datetime(newData["end"])
    
    del newData[0]
    newData = newData.reset_index(drop=True)
    
    return newData

newData = OptimizeCalendar("../datasets/calendar08.csv")
newData

,start,end,listing_id,available,num_day,minimum_nights,maximum_nights
0,2020-09-13,2020-11-30,2577,f,79,3.0,1125.0
1,2020-12-01,2021-08-30,2577,t,273,3.0,1125.0
2,2021-08-31,2021-08-31,2577,f,1,3.0,1125.0
3,2021-09-01,2021-09-12,2577,t,12,3.0,1125.0
4,2020-09-13,2020-09-20,3109,f,8,2.0,30.0
...,...,...,...,...,...,...,...
207191,2020-10-16,2020-10-31,45395372,t,16,4.0,15.0
207192,2020-11-01,2020-11-30,45395372,f,30,4.0,15.0
207193,2020-12-01,2020-12-11,45395372,t,11,4.0,15.0
207194,2020-12-12,2021-09-12,45395372,f,275,4.0,15.0
